In [1]:
!pip install -U transformers
!pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 19.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.3
    Uninstalling transformers-4.43.3:
      Successfully uninstalled transformers-4.43.3
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 KB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 9.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Consider a

In [2]:
import numpy as np
import pandas as pd
import operator
import os
import string
import re
import random
import sys
import platform

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from PIL import Image

from transformers import Blip2Processor, Blip2ForConditionalGeneration

from tqdm.auto import tqdm

/home/jinjinjara1022/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jinjinjara1022/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [52]:
device = "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [53]:
import os
import json

from torchvision.datasets import ImageFolder
from natsort import natsorted
from datetime import datetime

In [54]:
with open(os.path.join(root, 'classes.json'), 'r') as j:
    class_names = json.loads(j.read())

In [55]:
class_names

['Buildings', 'Forests', 'Glacier', 'Mountains', 'Sea', 'Street']

In [103]:
img_path = f"./Scene/0/{238}.jpg"

In [109]:
image = Image.open(img_path).convert('RGB')
prompt = "Describe the scene: Is it a building, forest, glacier, mountain, sea, or street?"

In [110]:
inputs = processor(images=image, text=prompt, return_tensors="pt")

In [111]:
generated_ids = model.generate(**inputs)

In [112]:
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

In [113]:
generated_text

'mountain'

In [119]:
def text2index(gen_text):
    for idx, text in enumerate(['building', 'forest', 'glacier', 'mountain', 'sea', 'street']):
        if text == gen_text:
            return idx

    for idx, text in enumerate(['building', 'forest', 'glacier', 'mountain', 'sea', 'street']):
        if text in gen_text:
            return idx
    return -1

In [ ]:
img_nums = 8100

submission = dict({'id_idx': list(range(img_nums)), 'label': []})

for idx in range(img_nums):
    if (idx+1)%100 == 0:
        print(idx+1)
    img_path = f"./Scene/0/{idx}.jpg"
    image = Image.open(img_path).convert('RGB')
    prompt = "Describe the scene: Is it a building, forest, glacier, mountain, sea, or street?"
    inputs = processor(images=image, text=prompt, return_tensors="pt")
    generated_ids = model.generate(**inputs, max_new_tokens=20)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    #print(generated_text)
    submission['label'] += [text2index(generated_text)]

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300


In [126]:
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f'submission_{current_time}.csv'
pd.DataFrame(submission).to_csv(os.path.join(root+"submissions/", file_name), index=False)